#### Simple Gen AI APP Using Langchain

In [2]:
import os
from dotenv import load_dotenv
load_dotenv()

os.environ['OPENAI_API_KEY']=os.getenv("OPENAI_API_KEY")
## Langsmith Tracking
os.environ["LANGCHAIN_API_KEY"]=os.getenv("LANGCHAIN_API_KEY")
os.environ["LANGCHAIN_TRACING_V2"]="true"
os.environ["LANGCHAIN_PROJECT"]=os.getenv("LANGCHAIN_PROJECT")

In [3]:
## Data Ingestion--From the website we need to scrape the data
from langchain_community.document_loaders import WebBaseLoader

In [4]:
loader=WebBaseLoader("https://en.wikipedia.org/wiki/Lionel_Messi")
loader

In [5]:
docs=loader.load()
docs

[Document(page_content='\n\n\n\nLionel Messi - Wikipedia\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nJump to content\n\n\n\n\n\n\n\nMain menu\n\n\n\n\n\nMain menu\nmove to sidebar\nhide\n\n\n\n\t\tNavigation\n\t\n\n\nMain pageContentsCurrent eventsRandom articleAbout WikipediaContact us\n\n\n\n\n\n\t\tContribute\n\t\n\n\nHelpLearn to editCommunity portalRecent changesUpload fileSpecial pages\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nSearch\n\n\n\n\n\n\n\n\n\n\n\nSearch\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nAppearance\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nDonate\n\nCreate account\n\nLog in\n\n\n\n\n\n\n\n\nPersonal tools\n\n\n\n\n\nDonate Create account Log in\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nContents\nmove to sidebar\nhide\n\n\n\n\n(Top)\n\n\n\n\n\n1\nEarly life\n\n\n\n\n\n\n\n\n2\nYouth career\n\n\n\n\nToggle Youth career subsection\n\n\n\n\n\n2.1\nNewell\'s Old Boys\n\n\n\n\n\n\n\n\n2.2\nBarcelona\n\n\n\n\n\n\n\n\n\n\n3\nClub career\n\n\n\n\nTogg

In [6]:
### Load Data--> Docs-->Divide our Docuemnts into chunks dcouments-->text-->vectors-->Vector Embeddings--->Vector Store DB
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter=RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=200)
documents=text_splitter.split_documents(docs)

In [7]:
documents

[Document(page_content="Lionel Messi - Wikipedia\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nJump to content\n\n\n\n\n\n\n\nMain menu\n\n\n\n\n\nMain menu\nmove to sidebar\nhide\n\n\n\n\t\tNavigation\n\t\n\n\nMain pageContentsCurrent eventsRandom articleAbout WikipediaContact us\n\n\n\n\n\n\t\tContribute\n\t\n\n\nHelpLearn to editCommunity portalRecent changesUpload fileSpecial pages\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nSearch\n\n\n\n\n\n\n\n\n\n\n\nSearch\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nAppearance\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nDonate\n\nCreate account\n\nLog in\n\n\n\n\n\n\n\n\nPersonal tools\n\n\n\n\n\nDonate Create account Log in\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nContents\nmove to sidebar\nhide\n\n\n\n\n(Top)\n\n\n\n\n\n1\nEarly life\n\n\n\n\n\n\n\n\n2\nYouth career\n\n\n\n\nToggle Youth career subsection\n\n\n\n\n\n2.1\nNewell's Old Boys\n\n\n\n\n\n\n\n\n2.2\nBarcelona\n\n\n\n\n\n\n\n\n\n\n3\nClub career\n\n\n\n\nToggle Club c

In [8]:
## Creating Embeddings
from langchain_openai import OpenAIEmbeddings
embeddings=OpenAIEmbeddings()

In [9]:
## Store embeddings in FAISS Vector DB
from langchain_community.vectorstores import FAISS
vectorstoredb=FAISS.from_documents(documents,embeddings)

In [10]:
vectorstoredb

In [11]:
## Create llm 
from langchain_openai import ChatOpenAI
llm=ChatOpenAI(model="gpt-4o")

In [12]:
## Create Document chain

from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

prompt=ChatPromptTemplate.from_template(
    """
Answer the following question based on the provided context.
<context>
{context}
</context>
"""
)

document_chain=create_stuff_documents_chain(llm,prompt) ## prompt here contains the context that is passed to the llm. 
document_chain

RunnableBinding(bound=RunnableBinding(bound=RunnableAssign(mapper={
  context: RunnableLambda(format_docs)
}), config={'run_name': 'format_inputs'})
| ChatPromptTemplate(input_variables=['context'], messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context'], template='\nAnswer the following question based on the provided context.\n<context>\n{context}\n</context>\n'))])
| ChatOpenAI(client=<openai.resources.chat.completions.completions.Completions object at 0x117a41d20>, async_client=<openai.resources.chat.completions.completions.AsyncCompletions object at 0x117a423e0>, model_name='gpt-4o', openai_api_key=SecretStr('**********'), openai_proxy='')
| StrOutputParser(), config={'run_name': 'stuff_documents_chain'})

In [13]:
## Create Retriever Chain from Vector Store DB which contains the embeddings for the given documents
from langchain.chains import create_retrieval_chain

retriever=vectorstoredb.as_retriever()
retrieval_chain=create_retrieval_chain(retriever,document_chain)

In [14]:
retrieval_chain

RunnableBinding(bound=RunnableAssign(mapper={
  context: RunnableBinding(bound=RunnableLambda(lambda x: x['input'])
           | VectorStoreRetriever(tags=['FAISS', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x117a1cc10>), config={'run_name': 'retrieve_documents'})
})
| RunnableAssign(mapper={
    answer: RunnableBinding(bound=RunnableBinding(bound=RunnableAssign(mapper={
              context: RunnableLambda(format_docs)
            }), config={'run_name': 'format_inputs'})
            | ChatPromptTemplate(input_variables=['context'], messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context'], template='\nAnswer the following question based on the provided context.\n<context>\n{context}\n</context>\n'))])
            | ChatOpenAI(client=<openai.resources.chat.completions.completions.Completions object at 0x117a41d20>, async_client=<openai.resources.chat.completions.completions.AsyncCompletions object at 0x117a42

In [15]:
## Get the response form the LLM using the retriever chain
response=retrieval_chain.invoke({"input":"Tell me about Messi's football debut."})

In [16]:

response['answer']

"Based on the provided context, here are some key points about Lionel Messi's early career:\n\n1. **First-Team Debut**: Messi made his first-team debut for Barcelona at the age of 16 years, four months, and 23 days during a friendly match against José Mourinho's Porto on November 16, 2003. \n\n2. **Training and Integration**: Following his impressive debut, Messi began training daily with Barcelona B (the reserve team) and weekly with the first team. Ronaldinho, a star player at the time, recognized Messi's potential and supported his integration into the first team.\n\n3. **Youth Career**: Before his first-team debut, Messi progressed rapidly through Barcelona's youth ranks, debuting for four different youth teams during the 2003–04 season. He was especially notable in the Juvenil A team, where he scored 18 goals in 11 league games.\n\n4. **International Debut and Achievements**: Messi debuted with Argentina's senior national team in 2005 at age 18 during a friendly against Hungary. H

In [17]:
response['context']

[Document(page_content='At 16 years, four months, and 23 days old, Messi made his first-team debut when he came on in the 75th minute during a friendly against José Mourinho\'s Porto on 16 November 2003.[42][55] His performance, creating two chances and a shot on goal, impressed the technical staff, and he subsequently began training daily with the club\'s reserve side, Barcelona B, as well as weekly with the first team.[56] After his first training session with the senior squad, Barça\'s new star player, Ronaldinho, told his teammates that he believed the 16-year-old would become an even better player than himself.[57] Ronaldinho soon befriended Messi, whom he called "little brother". This greatly eased Messi\'s transition into the first team.[58][59]', metadata={'source': 'https://en.wikipedia.org/wiki/Lionel_Messi', 'title': 'Lionel Messi - Wikipedia', 'language': 'en'}),
 Document(page_content="Messi made his debut with the senior national team in 2005, at age 18, coming off the be